In [1]:
import gurobipy as gp
from gurobipy import GRB
import networkx as nx
import itertools
import geopandas
import geopy.geocoders as gg

In [2]:
# Create Directed Graph from n nodes. n must match the dimension of the distance matrix (n x n matrix)
n = 35
G = nx.DiGraph()

G.add_nodes_from(range(n))

for i in range(n):
    for j in range(n):
        G.add_edge(i, j)

In [3]:
# Parse through distance matrix file and add distance attributes to all edges

file = open("state parks distance matrix.txt", "r")
i = 0
j = 0
for line in file:
    for word in line.split():
        if word.find(",") >= 0:
            word = word[:word.index(",")]
        G.edges[i, j]["Distance"] = float(word)
        j += 1
    j = 0
    i += 1

In [4]:
m = gp.Model()

# Is the specified edge used? Yes "1" no "0"
isUsed = m.addVars(G.nodes, G.nodes, vtype=GRB.BINARY)
order = m.addVars(G.nodes, vtype=GRB.INTEGER)

# Define k
k = G.number_of_nodes()

#Create list of edge combinations for use in constraints
combos = list(itertools.combinations(range(k), 2))

Academic license - for non-commercial use only - expires 2021-05-15
Using license file C:\Users\Kyle\gurobi.lic


In [5]:
# Minimize the total route distance. Calculated by the distance of each node times the binary isUsed
# (whether the edge being referenced is chosen for the route "1", or not "0")

m.setObjective(gp.quicksum((G[i][j]["Distance"])*(isUsed[i, j]) for i in G.nodes for j in G.nodes), GRB.MINIMIZE)

# For all edges going towards a specific node, exactly one must be used.

m.addConstrs(gp.quicksum(isUsed[i, u] for i in G.nodes) == 1 for u in range(k))

# For all edges going out of a specific node, exactly one must be used. 

m.addConstrs(gp.quicksum(isUsed[u, j] for j in G.nodes) == 1 for u in range(k))

# Don't use the trivial edges created by the distance matrix. For example edge (1, 1) has a distance of 0 and 
# made no progress in the problem. 

m.addConstrs(isUsed[i, i] == 0 for i in G.nodes)

#MTZ ordering constraints

m.addConstrs(order[i] + isUsed[i, j] <= order[j] + (k - 1)*(1 - isUsed[i, j]) for i in G.nodes for j in range(1, k))
m.addConstr(order[0] == 0)

m.update()
#m.addConstr(gp.quicksum(isUsed[i, j] for i in subtour1 for j in subtour1) <= len(subtour1) - 1)

In [6]:
m.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1296 rows, 1260 columns and 5988 nonzeros
Model fingerprint: 0xda6c5001
Variable types: 0 continuous, 1260 integer (1225 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [2e+00, 6e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 70 rows and 36 columns
Presolve time: 0.01s
Presolved: 1226 rows, 1224 columns, 5814 nonzeros
Variable types: 0 continuous, 1224 integer (1190 binary)

Root relaxation: objective 1.747987e+03, 148 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1747.98692    0   77          - 1747.98692      -     -    0s
     0     0 1930.66975    0   98          - 1930.66975      -     -    0s
     0     0 1935

In [7]:
# Print results

for i in range(k):
    for j in range(k):
        if(isUsed[i, j].x == 1):
            print("Edge", i, "to", j, "is used")

Edge 0 to 3 is used
Edge 1 to 5 is used
Edge 2 to 7 is used
Edge 3 to 2 is used
Edge 4 to 25 is used
Edge 5 to 33 is used
Edge 6 to 21 is used
Edge 7 to 27 is used
Edge 8 to 23 is used
Edge 9 to 11 is used
Edge 10 to 13 is used
Edge 11 to 10 is used
Edge 12 to 14 is used
Edge 13 to 32 is used
Edge 14 to 9 is used
Edge 15 to 6 is used
Edge 16 to 34 is used
Edge 17 to 26 is used
Edge 18 to 16 is used
Edge 19 to 20 is used
Edge 20 to 24 is used
Edge 21 to 19 is used
Edge 22 to 29 is used
Edge 23 to 0 is used
Edge 24 to 28 is used
Edge 25 to 12 is used
Edge 26 to 31 is used
Edge 27 to 15 is used
Edge 28 to 1 is used
Edge 29 to 18 is used
Edge 30 to 8 is used
Edge 31 to 30 is used
Edge 32 to 17 is used
Edge 33 to 22 is used
Edge 34 to 4 is used
